In [268]:
from bs4 import BeautifulSoup
import urllib.request as req
import urllib.parse as par
from urllib.parse import quote

from selenium import webdriver
from selenium.webdriver.common.by import By
from chromedriver_autoinstaller import install

import numpy as np
import pandas as pd

from tqdm import tqdm

In [269]:
result_dictionary = {'merchandise_name':[],                        # ok

          # vendor, membership 통합됨
          'vendor_membership_name':[],                                  

          'delivery_fee':[],                            # ok
          'price_with_delivery_fee':[],                 # price_without_delivery_fee에 delivery_fee 합산해서 진행 예정
          'price_without_delivery_fee':[],              # ok
          'price_with_cardDiscount':[],
          'price_without_cardDiscount':[]
}

In [270]:
chromeDriver_dir = '/Users/bagdaehui/Desktop/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(chromeDriver_dir, options=options)

/var/folders/hg/t_3ps3k16ml5zf5b9hdjjpzw0000gn/T/ipykernel_1421/2628198429.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromeDriver_dir, options=options)


In [271]:
def make_url_address(merchandise, keyword):
    query = quote(merchandise+keyword)
    mer_query = quote(merchandise)
    url = f'https://search.shopping.naver.com/search/all?frm=NVSHMDL&iq={query}&origQuery={mer_query}&pagingIndex=1&pagingSize=40&productSet=model&query={mer_query}&sort=rel&timestamp=&viewType=list'
    return url

In [272]:
class Soup_SearchList(): 
    class_xpath = '.basicList_title__3P9Q7 > .basicList_link__1MaTN'

    def make_soup(self, url):
        self.response = req.urlopen(url)
        self.soup = BeautifulSoup(self.response, 'html.parser')
        return self.soup

    def make_search_list(self, soup):
        self.raw_list = soup.select(Soup.class_xpath)
        self.search_list = []

        for idx in range(len(self.raw_list)):
            self.search_results_dict = {}
            self.search_results_dict['title'] = self.raw_list[idx]['title']
            self.search_results_dict['link'] = self.raw_list[idx]['href']
            self.search_list.append(self.search_results_dict)
        return self.search_list
    

In [273]:
class NAVER_driver(Soup_SearchList):
    def __init__(self, merchandise, keyword):
        self.merchandise = merchandise
        self.keyword = keyword
        self.url = make_url_address(merchandise, keyword)
        

    
    def __NAVER_chrome_driver__(self):
        self.chromeDriver_dir = '/Users/bagdaehui/Desktop/chromedriver'
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.driver = webdriver.Chrome(chromeDriver_dir, options=self.options)
        return self.driver


In [274]:
def make_dataframe(search_list, result_dictionary):
    for idx in tqdm(range(len(search_list)), ncols=100):
        merchandise_name = search_list[idx]['title']

        child_url = search_list[idx]['link']
        child_soup = Soup_SearchList.make_soup(Soup_SearchList,child_url)

        vendor_membership_name = child_soup.select('.productList_mall_link__1-Q4X')
        for idx, name in enumerate(vendor_membership_name):
            result_dictionary['merchandise_name'].append(merchandise_name)
            result_dictionary['vendor_membership_name'].append(name.text)

            driver = NAVER_driver.__NAVER_chrome_driver__(NAVER_driver)
            driver.get(child_url)

            price_without_delivery_fee = driver.find_element(By.XPATH,
                f'//*[@id="section_price"]/div[3]/ul/li[{idx+1}]/div/div[3]/a/span'
            )
            result_dictionary['price_without_delivery_fee'].append(price_without_delivery_fee.text)

            delivery_fee = driver.find_element(By.XPATH,
                f'//*[@id="section_price"]/div[3]/ul/li[{idx+1}]/div/div[3]/div[2]'
            )
            if delivery_fee.text == '':
                delivery_fee = driver.find_element(By.XPATH,
                    f'//*[@id="section_price"]/div[3]/ul/li[{idx+1}]/div/div[3]/div[1]'
                )
            result_dictionary['delivery_fee'].append(delivery_fee.text)
    return result_dictionary

In [275]:
a = NAVER_driver('삼다수', '500ml')

In [276]:
search_results_list = a.make_search_list(a.make_soup(a.url))

In [277]:
search_results_list

[{'title': '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  'link': 'https://cr.shopping.naver.com/adcr.nhn?x=y1WVDD7Mr5%2B1AnyAlNBcK%2F%2F%2F%2Fw%3D%3DsF%2FWEicrUfwC6%2FY7tH1%2BwTWIFiwNa%2FFBMZsYaCV57cBMpDQHhA9574RbpM0JOv6aBagbNoLxREbq6wNZRzPwdLmMAUm9QbHqI6CpbiuBMrEho3mBGiaRYzofYMW%2FUNHbTqZZBKQLcllrGrkgMVb58Q8OmY27ZGze0Q%2BtXRlM4OxtFSICBvI1ATDFN58yC%2Fe2xnIyVtfXz4IyqPYRG9GN1N%2BJqodzubnreGIhp8MZNj2cBmnjZQfS5VNpyTjRfpKZ8AK1BccckEgXtMhHLM882bkPg%2F6PYJW9LpCSumzPJoigK7CtQzZWLlur1%2FFge4%2BEpkiP7OAFS1o80l%2Fzib%2FH9kcTImYskNTV4KnNTNJFMOvnqV63sjhhfyP%2F8zxw6dNpe00U%2Fu4uSgPh%2Fb3F3nr3raQZUmNA4gxLTebXdgCfMUNDLTomfrXgOU2qVhq7nJE0fc0wJztCB%2B4TM7xm1XvIh2PRYFy1tvB0Bu0B9ltsOr9YNZPRiPPcF56modHkqmWSwF69cyIWyHOqByujnY1CmbFf3rriHxu2QAwwkkh%2Fd3HeOv%2B%2BG7P8zZs54SDuTec41Esui%2BcDv7TaMyyWtifXYWm1LrZA%2FoXlKGFgf7r%2F5%2FxmFbDf4gHI5rSMJU1BWcIOg&nvMid=28856087586&catId=50002032'},
 {'title': '제주특별자치도개발공사 삼다수 500ml',
  'link': 'https://cr.shopping.naver.com/adcr.nhn?x=%2F0S7qn8%2BN%2BM3Y0PcNhesOP%2F%2F%2Fw%3D%3DsSz

In [278]:
test = make_dataframe(search_results_list, result_dictionary)

  0%|                                                                         | 0/3 [00:00<?, ?it/s]/var/folders/hg/t_3ps3k16ml5zf5b9hdjjpzw0000gn/T/ipykernel_1421/915364426.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(chromeDriver_dir, options=self.options)
100%|█████████████████████████████████████████████████████████████████| 3/3 [01:48<00:00, 36.30s/it]


In [279]:
test

{'merchandise_name': ['제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 제주삼다수 그린 무라벨 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다수 500ml',
  '제주특별자치도개발공사 삼다

In [282]:
len(test['merchandise_name'])

43

In [283]:
len(test['vendor_membership_name'])

43